# DATA PREPARATOR FOR FL 32 dataset

importing need libraries

In [3]:
from __future__ import print_function, division
from skimage import io, transform
from random import randint
import shutil

import os, uuid, glob, warnings
import numpy as np

warnings.filterwarnings("ignore")

#### PATHES FOR FILES IN FL32

In [82]:
ORIGINAL_DATA_DIR = '../data/fl32/originals'
CROPPED_DATA_DIR = '../data/fl32/images'
ORIGINAL_ANNOTATION = '../data/fl32/annotation.txt'
CROPPED_ANNOTATION = '../data/fl32/crop_annotation.txt'

TRAIN_SET = '../annotations/trainset32.txt'
TEST_SET = '../annotations/testset32.txt'


['x y width height\r\n', '89 213 134 152\r\n']

Создание annotation.txt файл с названиями, классами и их позициями

Так же нужно запустить copy_no_logo.sh файл для того что скопировать изображения без логотипов

In [47]:
path = '../data/fl32/classes/masks'
f1 = open(ORIGINAL_ANNOTATION,'w')
for root, dirs, files in os.walk(path):
    if(len(files)!=0):
        for item in files:
            f = open(os.path.join(root,item), 'r')
            content = f.readlines()[1:]
            content = [x.split('\r\n')[:-1] for x in content]
            content = [x[0].split(' ') for x in content]
            for line in content:
                a = item.split('.bboxes.txt')[0]
                b = root.split('/')[-1]
                c = line
                into = '{} {} {} \n'.format(a,b,' '.join(c))
                f1.write(into)
            f.close()
f1.close()

Reading from annotation file

In [52]:
def read_from_annotation(path):
    file = open(path, "r")
    content = file.readlines()
    new = [x.split(" ")[:-1] for x in content]
    return new

# x = read_from_annotation(ORIGINAL_ANNOTATION)
# print(len(x))

Preparing labels array

In [55]:
def prepare_labels(annotaton_path):
    arr = read_from_annotation(annotaton_path)
    labels = []
    for item in (arr):
        l = item[1]
        if l not in labels:
            labels.append(l)
        else:
            continue
    return labels
LABELS = prepare_labels(ORIGINAL_ANNOTATION)
print(LABELS)
print(len(LABELS))

['ferrari', 'ups', 'cocacola', 'guiness', 'adidas', 'aldi', 'texaco', 'nvidia', 'rittersport', 'paulaner', 'dhl', 'bmw', 'fosters', 'milka', 'starbucks', 'pepsi', 'singha', 'apple', 'fedex', 'carlsberg', 'hp', 'chimay', 'google', 'tsingtao', 'corona', 'ford', 'esso', 'shell', 'stellaartois', 'becks', 'heineken', 'erdinger']
32


Creating cropped resized images
Run delete_cropped_images.sh file deleting last files from images directory 

In [74]:
def create_resized_crops(input_annotation, input_directory,
                        output_annotation, output_directory):
    annotation = read_from_annotation(input_annotation)
    output_annotation = open(output_annotation, 'w')
    for i, data in zip(range(len(annotation)),annotation):
        img_name = data[0]
        img_path = os.path.join(input_directory, img_name)
        image = io.imread(img_path)
        positions = data[-4:]
        x1 = int(positions[0])
        y1 = int(positions[1])
        x2 = int(positions[2])
        y2 = int(positions[3])

        if x1 > x2:
            tmp = x1
            x1 = x2
            x2 = tmp

        if y1 > y2:
            tmp = y1
            y1 = y2
            y2 = tmp
        
        
            
        target = data[1]
#         fl32 - dataset
        crop_resizer(image, target, x1,x1+x2,y1,y1+y2,output_annotation, output_directory)
# 
#         fl27 - dataset
#         crop_resizer(image, target, x1,x2,y1,y2,output_annotation, output_directory)
#         crop_resizer(image, target, x1-40,x2,y1,y2,output_annotation, output_directory)
#         crop_resizer(image, target, x1,x2+40,y1,y2,output_annotation, output_directory)
#         crop_resizer(image, target, x1,x2,y1-40,y2,output_annotation, output_directory)
#         crop_resizer(image, target, x1,x2,y1,y2+40,output_annotation, output_directory)

def crop_resizer(image,target, x1,x2,y1,y2, writer,directory):
    crop_img = image[y1:y2, x1:x2]
    file_name = str(uuid.uuid4().hex)+'.jpg'
    try:
        try:
            resized = transform.resize(crop_img,(480, 480))
            io.imsave(os.path.join(directory, file_name), resized)
            note = "{} {} \n".format(file_name, target)
            writer.write(note)
        except ValueError:
            pass
    except IndexError:
        pass
create_resized_crops(ORIGINAL_ANNOTATION, ORIGINAL_DATA_DIR, CROPPED_ANNOTATION, CROPPED_DATA_DIR)

In [64]:
NO_LOGO_DIRECTORY = '../data/fl32/classes/no-logo'

In [69]:
NO_LOGO_DATA = glob.glob(os.path.join(NO_LOGO_DIRECTORY,'*.jpg'))
len(NO_LOGO_DATA)

6000

Запускать только под предлогом смерти

In [76]:
images = []
for item in NO_LOGO_DATA:
    img_name = item.split('/')[-1]
    im = io.imread(item)
    resized = transform.resize(im,(480, 480))
    io.imsave( os.path.join(CROPPED_DATA_DIR,img_name), resized)
    images.append([img_name, 'nologo'])

In [77]:
len(images)

6000

In [81]:
no_logo_annotation = '../data/fl32/nologo.txt'
f2 = open(no_logo_annotation, 'w')
for item in images:
    note = "{} {} \n".format(item[0], item[1])
    f2.write(note)
f2.close()

In [83]:
import random
def train_test_split(crop_ann_path,  test_size = 1000):
    train = read_from_annotation(crop_ann_path)
    test = []
    for i in range(test_size):
        randIndex = randint(0,len(train)-1)
        test.append(train[randIndex])
        del(train[randIndex])
    
    print(len(train))
    print(len(test))
    random.shuffle(train)
    
    
    trainset = open(TRAIN_SET, "w")
    testset = open(TEST_SET, "w")
    
    for item in train:
        tmp = "{} {} \n".format(item[0], item[1])
        trainset.write(tmp)
    trainset.close()

    for item in test:
        tmp = "{} {} \n".format(item[0], item[1])
        testset.write(tmp)
    testset.close()
train_test_split(CROPPED_ANNOTATION,2000)

7404
2000
